In [ ]:
#
# Stub Bundling and Confluent Spirals for Geographic Networks
# - Arlind Nocaj and Ulrik Brandes
# - Department of Computer & Information Science, University of Konstanz
# S. Wismath and A. Wolff (Eds.): GD 2013, LNCS 8242, pp. 388–399, 2013.
#
from math import pi, tan, exp, cos, sin
import rtsvg
rt = rtsvg.RACETrack()
x0, y0, x1, y1 = -10.0, -10.0, 20.0, 20.0
ws, hs         = 512, 512
pw    = ( 3.0,  4.0)
pv    = (10.0,  5.0)
svg = [f'<svg x="0" y="0" width="{ws}" height="{hs}" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff" />')
x = int(x0)
while x < x1:
    svg.append(f'<line x1="{x}" y1="{y0}" x2="{x}" y2="{y1}" stroke="#a0a0a0" stroke-width="0.01"/>')
    x += 1
y = int(y0)
while y < y1:
    svg.append(f'<line x1="{x0}" y1="{y}" x2="{x1}" y2="{y}" stroke="#a0a0a0" stroke-width="0.01"/>')
    y += 1
svg.append(f'<circle cx="{pw[0]}" cy="{pw[1]}" r="0.2" fill="#ff0000"/>')
svg.append(f'<circle cx="{pv[0]}" cy="{pv[1]}" r="0.2" fill="#00ff00"/>')
svg_base = ''.join(svg)

In [ ]:
cot   = lambda x: 1.0 / tan(x)
a     = 2.4
theta = -pi / 4.0
b     = cot(theta)
def S(t):
    _in_exp_     = -1.0 * abs(b) * t
    _uv_         = rt.unitVector((pw, pv))
    _phi0_       = _uv_[0] * 1.0 + _uv_[1] * 0.0
    _in_cos_     = (b / abs(b)) * (_phi0_ + t)
    _in_sin_     = (b / abs(b)) * (_phi0_ + t)
    x = pw[0] + abs(pv[0] - pw[0]) * exp(_in_exp_) * cos(_in_cos_)
    y = pw[1] + abs(pv[1] - pw[1]) * exp(_in_exp_) * sin(_in_sin_)
    return (x, y)
svg = [svg_base]
t   = 0.0
while t < 10.0:
    x,y = S(t)
    svg.append(f'<circle cx="{x}" cy="{y}" r="0.05" fill="#000000"/>')
    t += 0.1
svg.append('</svg>')
rt.tile([''.join(svg)])